In [1]:
# carregar bibliotecas
source('fun/setup.R')

# Dados a atualizar de acordo com o mês a rodar
# ano_mes <- '201811'
# ano_mes <- '201812'
ano_mes <- '201901'

# Estrutura de pastas
pasta_dados        <- "../../yellow_dados"
pasta_modelos      <- sprintf('%s/06_bases_para_modelo', pasta_dados)
pasta_trechos_proc <- sprintf('%s/A_trechos_processados/%s', pasta_modelos, ano_mes)
pasta_base_agrup   <- sprintf('%s/B_processados_agrupados', pasta_modelos)
dir.create(pasta_base_agrup, recursive = TRUE, showWarnings = FALSE)

GDAL version >= 3.1.0 | setting mapviewOptions(fgb = TRUE)

Linking to GEOS 3.9.1, GDAL 3.2.2, PROJ 8.0.0


Attaching package: ‘jsonlite’


The following object is masked from ‘package:purrr’:

    flatten




In [2]:
# ----------------------------------------------------------
# Juntar e salvar resultados em arquivo único
# ----------------------------------------------------------

# Pegar nomes de todos os .csvs de resultado
result_files <- list.files(pasta_trechos_proc, 
                           pattern = '^\\d{6}_\\d{2}_agrupado.csv', 
                           recursive = TRUE, 
                           full.names = TRUE)

# Criar um dataframe único para todos os registros daquele dia
# https://stackoverflow.com/questions/46299777/add-filename-column-to-table-as-multiple-files-are-read-and-bound
result_files <- data.frame(f_path = result_files)

dim(result_files)
head(result_files)

[1] 58510     1

,f_path
,<chr>
1,../../yellow_dados/06_bases_para_modelo/A_trechos_processados/201901/000265_00_agrupado.csv
2,../../yellow_dados/06_bases_para_modelo/A_trechos_processados/201901/000266_00_agrupado.csv
3,../../yellow_dados/06_bases_para_modelo/A_trechos_processados/201901/000268_00_agrupado.csv
4,../../yellow_dados/06_bases_para_modelo/A_trechos_processados/201901/000271_01_agrupado.csv
5,../../yellow_dados/06_bases_para_modelo/A_trechos_processados/201901/000274_00_agrupado.csv
6,../../yellow_dados/06_bases_para_modelo/A_trechos_processados/201901/000275_00_agrupado.csv


In [3]:
# Processar todos os arquivos das linhas em um df único
tmp_df <- 
  result_files %>%
  # Abrir cada arquivo e extrair conteúdo em nova coluna
  mutate(reg = lapply(f_path, read_delim, delim = ';', col_types = cols(.default = "c"))) %>% 
  # Transformar cada linha de conteúdo em uma linha nova no tibble (linhas com
  # nomes de arquivos vão se repetir para cada linha de conteúdo)
  unnest(reg, keep_empty = TRUE) %>% 
  # Retirar primeira coluna, com referência aos endereços do arquivo
  select(2:17)

dim(tmp_df)
head(tmp_df)

mutate: new variable 'reg' (list) with 58,502 unique values and 0% NA

select: dropped one variable (f_path)



[1] 1537881      16

trip_id,edges.way_id,qgis_id,cluster,linha_sent,elev_sent,edges.length,n_pontos,ts_inicio,tempo_trecho,qgisid_ext_m,speed_kph,vel_med_gps,vel_med_m3,vel_med_m5,elev_grad_rev
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
000265_00,95944116,264627,1,des_linha,subida,53,2,1546423814.789,4.523999929428101,56.437,42.17506697090508,13.857031322093171,6.2067242445575985,0.754195899857389,0.5067597498096252
000265_00,229481094,264630,2,des_linha,descida,67,11,1546423822.119,57.24399995803833,66.969,4.2135420336944875,7.2031095433683285,8.61441269908937,10.1720778154487,-0.588331914766542
000265_00,760437724,053822,3,des_linha,descida,113,5,1546423884.318,20.01099991798401,113.229,20.32881923278638,15.986978532010834,14.166832778128926,15.808986137262224,-1.1384009396885844
000265_00,27467667,215993,4,des_linha,subida,51,2,1546423909.336,4.99399995803833,51.31,36.76411724923584,19.93144317498627,16.569353834377996,16.569353834377996,0.49892808419428086
000265_00,27467667,215994,5,des_linha,descida,55,2,1546423919.347,4.995000123977661,54.534,39.639638655769836,14.035612605072707,17.22801564741109,19.776166834780266,-0.2897275094436782
000265_00,27473198,142461,6,opo_linha,descida,213,12,1546423929.316,55.03099989891052,212.858,13.933964518336523,13.383454213458176,13.045710530573492,12.232963866970444,-0.51301806838361


In [4]:
# Definir arquivo de saída
out_file <- sprintf('%s/%s_trechos_processados_todos.csv', pasta_base_agrup, ano_mes)
write_delim(tmp_df, out_file, delim = ';')